In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pyrcn.echo_state_network import ESNRegressor

In [3]:
import glob
import warnings
warnings.filterwarnings("ignore")

# iterate through all tsv files
for filename in glob.glob("data/*.tsv"):
    list_to_compute = ["PAT14", "PAT15", "PAT16"]
    # compute only for files with those IDs
    if not any(x in filename for x in list_to_compute):
        continue

    try:
        print(filename)
        
        # load data
        df = pd.read_csv(filename, header=None)
        df = df.T

        num_of_regions = df.shape[0]
        train_len = int(df.shape[1] * 0.6)

        causality_matrix = np.zeros((num_of_regions, num_of_regions))
        for i in range(num_of_regions):
            for j in range(num_of_regions):
                if i == j:
                    causality_matrix[i, j] = 0
                    continue
                x = df.iloc[i]
                y = df.iloc[j]
                x = x.values.reshape(-1,1)
                y = y.values.reshape(-1,1)

                X_train = np.array(x[:train_len]).reshape(-1, 1)
                X_test = np.array(x[train_len:]).reshape(-1, 1)
                y_train = np.array(y[:train_len])
                y_test = np.array(y[train_len:])

                reg = ESNRegressor(hidden_layer_size=20)
                reg.fit(X=X_train, y=y_train)

                y_pred = reg.predict(X_test)
                test_corr = np.corrcoef(y_test.reshape(-1), y_pred.reshape(-1))[0, 1]
                causality_matrix[i][j] = test_corr
        
        # save causality matrix
        np.save(filename[:-4] + "_causality_matrix.npy", causality_matrix)

    except Exception as e:
        print(e)
        continue

data/sub-PAT15_ses-preop_task-rest_space-MNI152NLin2009cAsym_atlas-Gordon_desc-timeseries_bold.tsv
data/sub-PAT16_ses-preop_task-rest_space-MNI152NLin2009cAsym_atlas-Gordon_desc-timeseries_bold.tsv
data/sub-PAT14_ses-preop_task-rest_space-MNI152NLin2009cAsym_atlas-Gordon_desc-timeseries_bold.tsv
